In [1]:
!git clone https://github.com/nyu-mll/jiant.git
%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

Cloning into 'jiant'...
remote: Enumerating objects: 13802, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 13802 (delta 29), reused 60 (delta 14), pack-reused 13697
Receiving objects: 100% (13802/13802), 4.38 MiB | 22.65 MiB/s, done.
Resolving deltas: 100% (9635/9635), done.
/content/jiant
     |████████████████████████████████| 256kB 40.9MB/s 
     |████████████████████████████████| 6.3MB 40.8MB/s 
     |████████████████████████████████| 153kB 53.0MB/s 
     |████████████████████████████████| 1.5MB 42.7MB/s 
     |████████████████████████████████| 163kB 58.0MB/s 
     |████████████████████████████████| 20.2MB 1.3MB/s 
     |████████████████████████████████| 10.0MB 39.9MB/s 
Reason for being yanked: Insecure, upgrade to 0.12.1
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 890kB 48.8MB/s 
     |████████████████████████████████| 1.1MB 42.0MB/s 
     |█████████████████████

Obtaining file:///content/jiant
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Running setup.py develop for jiant


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Restart runtime after installing libs

I added some code to jiant source code in order to achieve the configuration of the number bert layers. So you must overwrite some jiant source codes (on colab) by uploading the following code in our repository:


*   jiant\jiant\proj\main\runscript.py **overwrites** jiant\jiant\proj\main\runscript.py
*   jiant\jiant\proj\main\modeling\model_setup.py **overwrites** jiant\jiant\proj\main\modeling\model_setup.py

This step above **must** be done before you continue

Added some code to realize the supporting facts probing task in jiant. Add the following file from our repository to the jiant installation:

* jiant\jiant\tasks\lib\templates\two_span_single_label.py **to** jiant/jiant/tasks/lib/templates/
*   jiant\jiant\tasks\lib\edge_probing\sup.py **to** jiant/jiant/tasks/edge_probing
* jiant\jiant\tasks\lib\templates\single_span_single_label.py **to** jiant/jiant/tasks/lib/templates/
*   jiant\jiant\tasks\lib\edge_probing\trec.py **to** jiant/jiant/tasks/edge_probing
*   jiant\jiant\tasks\lib\edge_probing\labels.txt **to** jiant/jiant/tasks/edge_probing
* jiant\jiant\tasks\retrieval.py **to** jiant\jiant\tasks\retrieval.py
* jiant\jiant\tasks\core.py **to** jiant\jiant\tasks\core.py
* jiant\jiant\tasks\evaluate\core.py **to** jiant\jiant\tasks\evaluate\core.py
* jiant\jiant\proj\main\modeling\taskmodels.py **to** jiant\jiant\proj\main\modeling\taskmodels.py
* jiant\jiant\proj\main\modeling\heads.py **to** jiant\jiant\proj\main\modeling\heads.py


# Question type probing task

In [1]:
task_name = "trec" # aka TREC: question type probing task

In [2]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

os.makedirs("/content/tasks/configs/", exist_ok=True)
os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

py_io.write_json({
  "task": task_name,
  "paths": {
    "train": f"/content/tasks/data/{task_name}/train_small.json",
    "val":   f"/content/tasks/data/{task_name}/val.json",
  },
  "name": task_name
}, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [3]:
import jiant.proj.main.export_model as export_model

model = "csarron/bert-base-uncased-squad-v1"  # BERT-base uncased model fine-tuned on SQuAD v1

export_model.export_model(
    hf_pretrained_model_name_or_path=model,
    output_base_path=f"/content/models/{model}"
)

Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at csarron/bert-base-uncased-squad-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.

You must then upload the files val.json and train_small.json  of the repository folder datasets/trec-10/output/ to the tasks/data/trec folder in colab. 

This step above **must** be done before you continue

In [ ]:
# this is for testing only
from jiant.tasks.lib.edge_probing.sup import TrecTask

task = TrecTask('test',
               {
                "train": f"/content/tasks/data/{task_name}/train_small.json",
                "val":   f"/content/tasks/data/{task_name}/val.json",
                })
task.get_train_examples()

In [4]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task

tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"/content/tasks/configs/{task_name}_config.json",
    hf_pretrained_model_name_or_path="bert-base-uncased",
    output_dir=f"/content/cache/{task_name}",
    phases=["train", "val"],
    max_seq_length=512
))

TrecTask
  [train]: /content/tasks/data/trec/train_small.json
  [val]: /content/tasks/data/trec/val.json


ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectur

In [5]:
import jiant.shared.caching as caching

row = caching.ChunkedFilesDataCache(f"/content/cache/{task_name}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[ 101 2073 2106 3021 6733 2175 2000 2267 1029  102    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [6]:
import jiant.proj.main.scripts.configurator as configurator

# Hyperparameters 
epochs = 5  # defined in paper

jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="/content/tasks/configs",
    task_cache_base_path="/content/cache",
    train_task_name_list=[task_name],
    val_task_name_list=[task_name],
    train_batch_size=8,
    eval_batch_size=2,
    epochs=epochs,
    num_gpus=1,
).create_config()
os.makedirs("/content/run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
display.show_json(jiant_run_config)

{
  "task_config_path_dict": {
    "trec": "/content/tasks/configs/trec_config.json"
  },
  "task_cache_config_dict": {
    "trec": {
      "train": "/content/cache/trec/train",
      "val": "/content/cache/trec/val",
      "val_labels": "/content/cache/trec/val_labels"
    }
  },
  "sampler_config": {
    "sampler_type": "ProportionalMultiTaskSampler"
  },
  "global_train_config": {
    "max_steps": 1075,
    "warmup_steps": 107
  },
  "task_specific_configs_dict": {
    "trec": {
      "train_batch_size": 8,
      "eval_batch_size": 2,
      "gradient_accumulation_steps": 1,
      "eval_subset_num": 500
    }
  },
  "taskmodels_config": {
    "task_to_taskmodel_map": {
      "trec": "trec"
    },
    "taskmodel_config_map": {
      "trec": null
    }
  },
  "task_run_config": {
    "train_task_list": [
      "trec"
    ],
    "train_val_task_list": [
      "trec"
    ],
    "val_task_list": [
      "trec"
    ],
    "test_task_list": []
  },
  "metric_aggregator_config": {
    "metri

In [ ]:
# You may restart runtime when you see error

import jiant.proj.main.runscript as main_runscript

# Hyperparameters 
eval_every_steps = 1000 # defined in paper
num_layers = 5          # change it to test

run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
    output_dir=f"/content/runs/{task_name}",
    hf_pretrained_model_name_or_path="csarron/bert-base-uncased-squad-v1",
    model_path="/content/models/csarron/bert-base-uncased-squad-v1/model/model.p",
    model_config_path="/content/models/csarron/bert-base-uncased-squad-v1/model/config.json",
    learning_rate=1e-5,
    eval_every_steps=eval_every_steps,
    do_train=True,
    do_val=True,
    do_save=True,
    force_overwrite=True,
    num_hidden_layers=num_layers,
)

main_runscript.run_loop(run_args)

  jiant_task_container_config_path: /content/run_configs/trec_run_config.json
  output_dir: /content/runs/trec
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 5
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False

/content/jiant/jiant/proj/main/modeling/model_setup.py:177: UserWarning: The following weights were not loaded: dict_keys(['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias'])
  "The following weights were not loaded: {}".format(remainder_weights_dict.keys())


No optimizer decay for:
  encoder.embeddings.LayerNorm.weight
  encoder.embeddings.LayerNorm.bias
  encoder.encoder.layer.0.attention.self.query.bias
  encoder.encoder.layer.0.attention.self.key.bias
  encoder.encoder.layer.0.attention.self.value.bias
  encoder.encoder.layer.0.attention.output.dense.bias
  encoder.encoder.layer.0.attention.output.LayerNorm.weight
  encoder.encoder.layer.0.attention.output.LayerNorm.bias
  encoder.encoder.layer.0.intermediate.dense.bias
  encoder.encoder.layer.0.output.dense.bias
  encoder.encoder.layer.0.output.LayerNorm.weight
  encoder.encoder.layer.0.output.LayerNorm.bias
  encoder.encoder.layer.1.attention.self.query.bias
  encoder.encoder.layer.1.attention.self.key.bias
  encoder.encoder.layer.1.attention.self.value.bias
  encoder.encoder.layer.1.attention.output.dense.bias
  encoder.encoder.layer.1.attention.output.LayerNorm.weight
  encoder.encoder.layer.1.attention.output.LayerNorm.bias
  encoder.encoder.layer.1.intermediate.dense.bias
  encode

# Supporting facts probing task

In [8]:
task_name = "sup" # aka SUP: supporting facts probing task

In [9]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

os.makedirs("/content/tasks/configs/", exist_ok=True)
os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

py_io.write_json({
  "task": task_name,
  "paths": {
    "train": f"/content/tasks/data/{task_name}/train.json",
    "val":   f"/content/tasks/data/{task_name}/val.json",
  },
  "name": task_name
}, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [10]:
import jiant.proj.main.export_model as export_model

model = "csarron/bert-base-uncased-squad-v1"  # BERT-base uncased model fine-tuned on SQuAD v1

export_model.export_model(
    hf_pretrained_model_name_or_path=model,
    output_base_path=f"/content/models/{model}"
)

Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at csarron/bert-base-uncased-squad-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.

You must then upload the *jsonl files of the repository folder datasets/squad1.1/small to the tasks/data/sup folder in colab. 

This step above **must** be done before you continue

In [4]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task

tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"/content/tasks/configs/{task_name}_config.json",
    hf_pretrained_model_name_or_path="bert-base-uncased",
    output_dir=f"/content/cache/{task_name}",
    phases=["train", "val"],
    max_seq_length=512
))

ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT
ModelArchitectures.BERT


KeyboardInterrupt: ignored

In [11]:
import jiant.shared.caching as caching

row = caching.ChunkedFilesDataCache(f"/content/cache/{task_name}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[ 101 2073 2106 3021 6733 2175 2000 2267 1029  102    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [12]:
import jiant.proj.main.scripts.configurator as configurator

# Hyperparameters 
epochs = 5  # defined in paper

jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="/content/tasks/configs",
    task_cache_base_path="/content/cache",
    train_task_name_list=[task_name],
    val_task_name_list=[task_name],
    train_batch_size=8,
    eval_batch_size=2,
    epochs=epochs,
    num_gpus=1,
).create_config()
os.makedirs("/content/run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
display.show_json(jiant_run_config)

{
  "task_config_path_dict": {
    "trec": "/content/tasks/configs/trec_config.json"
  },
  "task_cache_config_dict": {
    "trec": {
      "train": "/content/cache/trec/train",
      "val": "/content/cache/trec/val",
      "val_labels": "/content/cache/trec/val_labels"
    }
  },
  "sampler_config": {
    "sampler_type": "ProportionalMultiTaskSampler"
  },
  "global_train_config": {
    "max_steps": 1075,
    "warmup_steps": 107
  },
  "task_specific_configs_dict": {
    "trec": {
      "train_batch_size": 8,
      "eval_batch_size": 2,
      "gradient_accumulation_steps": 1,
      "eval_subset_num": 500
    }
  },
  "taskmodels_config": {
    "task_to_taskmodel_map": {
      "trec": "trec"
    },
    "taskmodel_config_map": {
      "trec": null
    }
  },
  "task_run_config": {
    "train_task_list": [
      "trec"
    ],
    "train_val_task_list": [
      "trec"
    ],
    "val_task_list": [
      "trec"
    ],
    "test_task_list": []
  },
  "metric_aggregator_config": {
    "metri

In [13]:
# You may restart runtime when you see error

import jiant.proj.main.runscript as main_runscript

# Hyperparameters 
eval_every_steps = 1000 # defined in paper
num_layers = 5          # change it to test

run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
    output_dir=f"/content/runs/{task_name}",
    hf_pretrained_model_name_or_path="csarron/bert-base-uncased-squad-v1",
    model_path="/content/models/csarron/bert-base-uncased-squad-v1/model/model.p",
    model_config_path="/content/models/csarron/bert-base-uncased-squad-v1/model/config.json",
    learning_rate=1e-5,
    eval_every_steps=eval_every_steps,
    do_train=True,
    do_val=True,
    do_save=True,
    force_overwrite=True,
    num_hidden_layers=num_layers,
)

main_runscript.run_loop(run_args)

  jiant_task_container_config_path: /content/run_configs/trec_run_config.json
  output_dir: /content/runs/trec
  hf_pretrained_model_name_or_path: csarron/bert-base-uncased-squad-v1
  model_path: /content/models/csarron/bert-base-uncased-squad-v1/model/model.p
  model_config_path: /content/models/csarron/bert-base-uncased-squad-v1/model/config.json
  model_load_mode: from_transformers
  num_hidden_layers: 5
  do_train: True
  do_val: True
  do_save: True
  do_save_last: False
  do_save_best: False
  write_val_preds: False
  write_test_preds: False
  eval_every_steps: 1000
  save_every_steps: 0
  save_checkpoint_every_steps: 0
  no_improvements_for_n_evals: 0
  keep_checkpoint_when_done: False
  force_overwrite: True
  seed: -1
  learning_rate: 1e-05
  adam_epsilon: 1e-08
  max_grad_norm: 1.0
  optimizer_type: adam
  no_cuda: False
  fp16: False
  fp16_opt_level: O1
  local_rank: -1
  server_ip: 
  server_port: 
device: cuda n_gpu: 1, distributed training: False, 16-bits training: False

KeyError: ignored